# Análisis para segunda pre entrega

## Analisis sobre disparidades regionales en salud de Argentina

### Objetivo del Proyecto

El objetivo de este trabajo será realizar un analisis de las disparidades regionales respecto los factores de riesgo como consumo de tabaco, alcohol, alimentación, actividad física, procesos de atención de salud y la existencia de enfermedades no transmisibles.
Se analizará por provincias y por regiones si existen diferencias significativas en la prevalencia de estas enfermedades no transmisibles como diabetes e hipertensión y las causas que llevan a estas diferencias (si existieran). Todo el análisis es válido para personas mayores de 18 años exclusivamente.

### Origen de los datos

Para el análisis se utilizarán los datos de la "Encuesta Nacional de Factores de Riesgo" realizada por el INDEC a adulos mayores de 18 años en Argentina.  
Para el análisis actual se utilizarán los resultados del año 2018:   
[Resultado encuesta](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.indec.gob.ar/ftp/cuadros/sociedad/cuadros_preliminares_enfr_2018.xls&ved=2ahUKEwiSjvbLl8yGAxUQppUCHbLeAQ4QFnoECDIQAQ&usg=AOvVaw2yheKKYGABUq8ZHkNbiPEI) 
[encuesta realizada]( https://www.indec.gob.ar/ftp/cuadros/sociedad/cuestionario_enfr_2018.pdf)



In [26]:
#importanción de librerías a utilizar

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sklearn as skl
from sklearn.preprocessing import MinMaxScaler
from colorama import init, Fore, Style

In [27]:
# IMPORTACIÓN DEL DATASET (resultado de la encuesta del año 2018)

friesgo_2018= pd.read_csv('friesgo2018.txt', delimiter = '|')

#vista rápida de las primeras 5 filas del dataset

friesgo_2018.head(5)

,id,cod_provincia,region,tamanio_aglomerado,aglomerado,localidades_150,submuestra,bhcv01,bhcv02,bhcv03,...,glucemia_elevada,prevalencia_glucemia_elevada_combinada,findrisc,bimq06,bimq06_01,colesterol_elevado,prevalencia_colesterol_combinada,wf1p,wf2p,wf3p
0,1128639,2,1,1,1,1,1,3,2,1,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2378,4464,0
1,1709939,2,1,1,1,1,0,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3588,0,0
2,6874130,2,1,1,1,1,0,3,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2740,0,0
3,10319375,2,1,1,1,1,1,3,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,493,0,0
4,11140857,2,1,1,1,1,0,3,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,0,0


In [35]:
#ANÁLISIS EXPLORATORIO SOBRE EL DATASET

print(f"{Style.BRIGHT}{Fore.BLUE}Número de filas y columnas del dataset\n{Style.RESET_ALL}")

print(f'{friesgo_2018.shape}\n')   # Cantidad de datos del dataset

print(f"{Style.BRIGHT}{Fore.BLUE}Número de datos nulos en cada columna\n{Style.RESET_ALL}")
print(f'{friesgo_2018.isnull().sum()}\n') #cantidad de nulos

print(f"{Style.BRIGHT}{Fore.BLUE}Porcentaje de datos nulos en cada columna\n{Style.RESET_ALL}")
print(f'{friesgo_2018.isna().sum()/friesgo_2018.shape[0] * 100}\n')   #porcentaje de nulos

print(f"{Style.BRIGHT}{Fore.BLUE}Nombre cada columna\n{Style.RESET_ALL}")
print(f'{friesgo_2018.columns}\n')   # tipos de variables de cada columna

print(f"{Style.BRIGHT}{Fore.BLUE}Tipos de datos de cada columna\n{Style.RESET_ALL}")
print(f'{friesgo_2018.dtypes}\n') # tipos de variables de cada columna

print(f"{Style.BRIGHT}{Fore.BLUE}Calculos estadísticos básicos sobre cada columna\n{Style.RESET_ALL}")
print(f"{friesgo_2018.describe(include='all')}\n")   #calculos estadísticos sobre datos incluye variables no num

print(f"{Style.BRIGHT}{Fore.BLUE}Información general sobre el dataset\n{Style.RESET_ALL}")
print(f'{friesgo_2018.info}\n')   #calculos estadísticos sobre datos

Número de filas y columnas del dataset

(29224, 287)

Número de datos nulos en cada columna

id                                      0
cod_provincia                           0
region                                  0
tamanio_aglomerado                      0
aglomerado                              0
                                    ...  
colesterol_elevado                  24313
prevalencia_colesterol_combinada    25821
wf1p                                    0
wf2p                                    0
wf3p                                    0
Length: 287, dtype: int64

Porcentaje de datos nulos en cada columna

id                                   0.000000
cod_provincia                        0.000000
region                               0.000000
tamanio_aglomerado                   0.000000
aglomerado                           0.000000
                                      ...    
colesterol_elevado                  83.195319
prevalencia_colesterol_combinada    88.355461
wf1p   


### Se observa que la cantidad de columnas presentes impiden que varios análisis sean útiles, por lo que se repetirá el análisis exploratorio SOLO sobre las columnas que se vayan a utilizar en este análisis.
### También se detectan varias columnas que tienen un conteo de cantidad de datos mucho menor a las 29224 filas que se expresan, por lo que una vez seleccionadas las columnas de interés se realizará una mayor limpieza ya que se debe determinar si los datos nulos son errores o son valores que no existen porque están respondidas en otra columna a continuación (por ejemplo si una columna es "tiene hipertensión" y la otra es "no tiene hipertensión" es claro que alguna de las dos no va a tener valor. 



In [46]:
print("mínima cantidad de valores en filas: ")
print(f"{friesgo_2018.count().min()}\n")
print("máxima cantidad de valores en filas: ")
print(f"{friesgo_2018.count().max()}\n")


conteos = friesgo_2018.count()

columnas_filtradas = conteos[conteos<29224].index.tolist()

print(columnas_filtradas)

mínima cantidad de valores en filas: 
17

máxima cantidad de valores en filas: 
29224

['bhcv10', 'bhcv11', 'bhho01', 'bhih01_02', 'rango_ingreso', 'imputado', 'bhch10_01_j', 'bhch10_02_j', 'bhch10_03_j', 'bhch10_04_j', 'bhch10_05_j', 'bhch10_06_j', 'bhch10_99_j', 'bhsl02', 'bhsl03', 'bhsl04', 'bhsl05', 'bhsl06', 'bhch10_01', 'bhch10_02', 'bhch10_03', 'bhch10_04', 'bhch10_05', 'bhch10_06', 'bhch10_99', 'bisl02', 'bisl03', 'bisl04', 'bisl05', 'bisl06', 'biaf02_m', 'biaf02_99', 'biaf04_m', 'biaf04_99', 'biaf06_m', 'biaf06_99', 'biaf07_m', 'biaf07_99', 'biaf08', 'biaf09', 'biaf10_01', 'biaf10_02', 'biaf10_03', 'biaf10_04', 'nivel_actividad_fisica', 'bita02', 'bita02_99', 'bita03', 'bita04', 'bita04_01', 'bita04_02', 'bita05', 'bita06_a', 'bita06_b', 'bita06_b_99', 'bita06_c', 'bita06_d', 'bita07', 'bita07_99', 'bita08', 'bita15', 'ta_paquete_y_armado', 'ta_dejar_fumar', 'hta_nofumadores', 'ta_perc_publicidad', 'ta_percepcion_riesgo', 'imagenes_tabaco', 'biha02', 'biha04', 'biha05_01', 'bi